# Projeto 1 - Ciência dos Dados

Nome: Fernando Bichuette Assumpção

Nome: Felipe Lacombe 

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [62]:
%matplotlib inline
import matplotlib.pyplot as plt
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
import emoji
import functools
import operator
from time import sleep
from random import shuffle

In [63]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Fernando\Documents\Insper\3° Semestre\Ciencia dos Dados\C_Dados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [64]:
#importando database
dados=pd.read_excel("bolsa.xlsx",'Teste')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

No projeto 1 de Ciência de dados será feito um clasificador que vai analizar a palavra bolsa em todos os tweets e classificar se é relevante quando está relacionada a **bolsa de valores** e irrelevante quando é empregada em outro em outro sentido ex: bolsa familia, bolsa utilitaria e etc.


## Limpando database

In [65]:
dados.B1.value_counts()

0    113
1     87
Name: B1, dtype: int64

In [66]:
#fazendo a limpeza do arquivo
def cleanup(tabela):
    tabela= tabela.replace("ç","c").replace("â","a").replace("ã","a").replace("à","a")\
    .replace("ê","e").replace("é","e").replace("ô","o").replace("õ","o").replace("/"," ")\
    .replace("?"," ").replace("!"," ").replace("@"," ").replace("\n"," ").replace(";"," ")\
    .replace(",","").replace(":"," ").replace("["," ").replace("]"," ").replace("}"," ")\
    .replace("{"," ").replace("\\"," ").replace(' " '," ").replace(" ' ","").replace(" * "," ")\
    .replace(" _ "," ").replace(" - "," ").replace(" . "," ").replace(" rt "," ").replace("…","")\
    .replace("“","").replace("             "," ").replace("   ","").replace(".."," ")\
    .replace("..."," ").replace("...."," ").replace("....."," ").replace("     "," ")\
    .replace("\ ","").replace("*","").replace(")","").replace(' " '," ").replace(" ' "," ")\
    .replace(' "'," ").replace('" '," ")\
    .replace('"'," ")
    
    
    return tabela 
titulo='Teste'
dados_palavras=dados['Teste'].apply(cleanup)

def separador(tabela,titulo):
    i = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['Tabela splitted'] = []

    for lista in i:
        for e in ['#','' , ' ', 'rt','@',]:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if palavra[:1] == "@":
                lista.remove(palavra)
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'Tabela splitted': lista}, ignore_index=True)
    return nova


def tabela_palavras(tabela):
    a = tabela['Tabela splitted']
    prob = pd.DataFrame()
    prob['Tabela splitted'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'Tabela splitted': e}, ignore_index=True)
    
    return prob['Tabela splitted']
dados_palavras=dados['Teste'].apply(cleanup)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [67]:
#Dataframe relevante 
relevantes = dados.loc[(dados['B1'])==1]
relevantes.set_index('B1',inplace=True)
relevantes.head()

,Teste
B1,
1,nos últimos 12 meses: \nibov vs cdi: cdi\nibov...
1,"após tombo de segunda, bolsa tem recuperação e..."
1,entenda por que a bolsa de valores cai e o dól...
1,bolsa de sao paulo suspende negocios tras desp...
1,tereza parou a aula pra falar do coronga e da ...


In [68]:
#Dataframe irrelevante
irrelevantes = dados.loc[(dados['B1'])==0]
irrelevantes.set_index('B1',inplace=True)
irrelevantes.head()

,Teste
B1,
0,agora sigo sem bolsa nenhuma pq as duas que eu...
0,entrei na mr cat hoje pra comprar uma bolsa de...
0,@zedassilva @zedassilva é um disparate sem so...
0,@_luizabatista bolsa de água quente ou banho b...
0,os crente gritando na minha cara aq na igreja ...


In [69]:
#Split os tweets dos dataframes criados acima
relevante_separado=separador(relevantes,titulo)
irrelevante_separado=separador(irrelevantes,titulo)

irrelevante_separado.head()

,Tabela splitted
0,"[agora, sigo, sem, bolsa, nenhuma, pq, as, dua..."
1,"[entrei, na, mr, cat, hoje, pra, comprar, uma,..."
2,"[@zedassilva, é, um, disparate, sem, sombra, d..."
3,"[bolsa, de, água, quente, ou, banho, bem, quen..."
4,"[os, crente, gritando, na, minha, cara, aq, na..."


In [70]:
#calculando quantas vezes cada palavra aparece nos tweets relevantes
splited_relevante=tabela_palavras(relevante_separado)
prob_relevante = splited_relevante.value_counts(True) 
num_palavras_relevante = splited_relevante.value_counts()
num_palavras_relevante.head()

de       81
a        77
bolsa    74
e        57
que      43
Name: Tabela splitted, dtype: int64

In [71]:
#calculando quantas vezes cada palavra aparece nos tweets irrelevantes
splited_irrelevante=tabela_palavras(irrelevante_separado)
prob_irrelevante = splited_irrelevante.value_counts(True) 
num_palavras_irrelevante = splited_irrelevante.value_counts()
num_palavras_irrelevante.head()

bolsa    102
de        82
e         77
a         56
que       51
Name: Tabela splitted, dtype: int64

In [72]:
# probabilidade de ocorrencia da palavra bolsa dentro do database relevante
prob_relevante['bolsa']

0.03722334004024145

In [73]:
#calculando a quantidade total de palavras temos em nosso database
tamanho=[]
newnova=nova.iloc[0:,0]
for t in newnova:
    for b in t:
        tamanho.append(b)

NameError: name 'nova' is not defined

In [ ]:
#criando novas colunas para o database final
nova["Classificacao Manual"] = pd.Series(dados["B1"])
nova["Classificacao_Naive"] = pd.Series()
nova["Resposta"] = pd.Series()

### Suavização de Laplace

In [ ]:
#criando um contador para percorrer o database linha por linha
a=0
#criando as variaveis que serão usadas na suavização de Laplace
total=len(tamanho)
total_relevantes=len(num_palavras_relevante)
total_irrelevantes=len(num_palavras_irrelevante)
len_01=len(num_palavras_relevante)
len_00=len(num_palavras_irrelevante)

#fazendo a suavização de Laplace
for valor in nova["Tabela splitted"]:
    alpha = 1
    prob01 = 1
    prob00 = 1
    for val in valor:
        #caso a palavra não exista no nosso database, terá seu valor alterado, para uma melhor 
        #classificação do Naive Bayers
        if val not in num_palavras_relevante or val not in num_palavras_irrelevante:
            sec01 = alpha
            sec00 = alpha
        
        else:
            sec00=num_palavras_irrelevante[val] + alpha
            sec01=num_palavras_relevante[val] + alpha
            
            
        if val in num_palavras_relevante:
            prob01 *= (num_palavras_relevante[val] + alpha) / (total_relevantes + total*alpha)
        else: 
            prob01 *= sec01/(len(num_palavras_relevante)+total*alpha)
        
        if val in num_palavras_irrelevante:   
            prob00 *= (num_palavras_irrelevante[val] + alpha) / (total_irrelevantes + total*alpha)
        else:
            prob00 *= sec00/(len(num_palavras_irrelevante)+total*alpha)
       
    if prob01>prob00:
        prob_x = 1
        nova["Classificacao_Naive"][a] = prob_x
        if nova["Classificacao Manual"][a] == prob_x:
            nova["Resposta"][a] = "Positivo = True"
        elif nova["Classificacao Manual"][a] != prob_x:
            nova["Resposta"][a] = "Positivo = False"
            
    elif prob01<prob00:
        prob_x = 0
        nova["Classificacao_Naive"][a] = prob_x
        if nova["Classificacao Manual"][a] == prob_x:
            nova["Resposta"][a] ="Negativo = True"
        elif nova["Classificacao Manual"][a] != prob_x:
            nova["Resposta"][a] ="Negativo = False"
    a+=1
resultado=nova

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
resultado.head()

In [ ]:
#checando a diferença entre as classificações feitas à mão e as definidas pelo Naive Byers
validador = pd.crosstab(resultado['Classificacao Manual'], resultado['Classificacao_Naive'])
validador.head()

In [ ]:
resposta = resultado["Resposta"].value_counts(True)
print(resposta*100)

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**